# 02 Extract Company report from FMP

In [1]:
from pyspark.sql.types import *
from pyspark.sql import functions as f
import requests
import pandas as pd 
from notebookutils import mssparkutils

In [2]:
# get data from api 
def get_financial_statement(symbol,limit,apikey):
    url ='https://financialmodelingprep.com/api/v3/financial-statement-full-as-reported/{0}?period=annual&limit={1}&apikey={2}'.format(symbol,limit,apikey)
    r = requests.get(url)
    data = r.json()
    return data

In [20]:
if __name__ == "__main__":
    
    # apikey = mssparkutils.credentials.getSecret('synapse-key12345','fmp-key')
    # # set up target ticker 
    # symbol_list=['NVDA','AMD','INTC','QCOM','GOOG','MSFT','AMZN','AAPL']
    # # get data from api
    # stock_data = [get_financial_statement(symbol,5,apikey) for symbol in symbol_list]
    # # list_ = [pd.DataFrame(data) for data in stock_data]

    selected_columns=[
            "date","symbol","period","documenttype","revenues","costofrevenue",'assetscurrent',
            "grossprofit","netincomeloss","earningspersharebasic","earningspersharediluted","operatingincomeloss",
            "operatingexpenses","researchanddevelopmentexpense","sellinggeneralandadministrativeexpense",
            "interestexpense","cashandcashequivalentsatcarryingvalue","commonstocksharesoutstanding","longtermdebt",
            'liabilitiescurrent','liabilities'
        ]
    list_ = [pd.DataFrame(data) for data in stock_data]
    pdfs = (pd.concat(list_ ))[[*selected_columns]]
    final_df_sp = spark.createDataFrame(pdfs)

    # change column name 
    final_df_sp = (
    final_df_sp
    .withColumnRenamed('costofrevenue','cost_of_revenue')
    .withColumnRenamed('grossprofit','gross_profit')
    .withColumnRenamed('netincomeloss','net_income_loss')
    .withColumnRenamed('netincomeloss','net_income_loss')
    .withColumnRenamed('earningspersharebasic','earnings_per_share_basic')
    .withColumnRenamed('earningspersharebasic','earnings_per_share_basic')
    .withColumnRenamed('earningspersharediluted','earnings_per_share_diluted')
    .withColumnRenamed('operatingincomeloss','operating_income_loss')
    .withColumnRenamed('operatingexpenses','operating_expenses')
    .withColumnRenamed('researchanddevelopmentexpense','research_development_expense')
    .withColumnRenamed('sellinggeneralandadministrativeexpense','selling_general_and_administrative_expense')
    .withColumnRenamed('interestexpense','interest_expense')
    .withColumnRenamed('cashandcashequivalentsatcarryingvalue','cash_and_cash_equivalents_at_carrying_value')
    .withColumnRenamed('commonstocksharesoutstanding','common_stock_shares_outstanding')
    .withColumnRenamed("longtermdebt",'long_term_debt')
    .withColumnRenamed("liabilitiescurrent",'liabilities_current')
    .withColumnRenamed("assetscurrent",'assets_current')
    )
    datalake_nm = 'datalake'+mssparkutils.env.getWorkspaceName()[7:] # get datalake name
    file_path ='abfss://files@{0}.dfs.core.windows.net/synapse/workspaces/data/company_report'.format(datalake_nm)
    final_df_sp.write.parquet(file_path,mode='overwrite')